In [1]:
from ucimlrepo import fetch_ucirepo 
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import cross_val_score
from sklearn.utils import shuffle
import featurize as ft
import numpy as np

np.random.seed(8888)

## Download data

In [2]:
# fetch dataset 
abalone = fetch_ucirepo(id=1) 
  
# data (as pandas dataframes) 
X = abalone.data.features 
y = abalone.data.targets["Rings"] 

for label in "MFI":
    X[label] = (X["Sex"] == label).astype(int)
del X["Sex"]

X.head()

,Length,Diameter,Height,Whole_weight,Shucked_weight,Viscera_weight,Shell_weight,M,F,I
0,0.455,0.365,0.095,0.5140,0.2245,0.1010,0.150,1,0,0
1,0.350,0.265,0.090,0.2255,0.0995,0.0485,0.070,1,0,0
2,0.530,0.420,0.135,0.6770,0.2565,0.1415,0.210,0,1,0
3,0.440,0.365,0.125,0.5160,0.2155,0.1140,0.155,1,0,0
4,0.330,0.255,0.080,0.2050,0.0895,0.0395,0.055,0,0,1


In [3]:
y.head()

0    15
1     7
2     9
3    10
4     7
Name: Rings, dtype: int64

## Cost Function

In [4]:
def cost_function(X, y):
    model = LinearRegression()
    scores = cross_val_score(model, X, y, cv=3, scoring="neg_mean_absolute_error")
    return scores.mean()

## Featurize

In [5]:
features = ft.featurize(
    X,
    y,
    selection_cost_func=cost_function,
    selection_bigger_is_better=True,
    n_jobs=-1,
    generate_parsimony_coefficient=0.01,
)

Optimising feature selection...:  24%|██▍       | 24/100 [00:02<00:09,  8.19it/s]


In [6]:
original = cost_function(X, y)
original

-1.6105265724876778

In [7]:
new = cost_function(features, y)
new

-1.5647407700313118

In [8]:
print(f"Old: {original}, New: {new}, Improvement: {round((1 - (new / original))* 100, 1)}%")

Old: -1.6105265724876778, New: -1.5647407700313118, Improvement: 2.8%
